In [3]:
import pandas as pd
import sys 
from bs4 import BeautifulSoup as bs

In [24]:

content = []
# Read the XML file
with open("export.xml", "r") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    bs_content = bs(content, "lxml")

data =[]
for rec in bs_content.find_all(attrs={"type" : "HKQuantityTypeIdentifierStepCount"}):
    r = [rec['startdate'],rec['enddate'],rec['value']]
    data.append(r)

    
cols =['startdate','enddate','value']

df = pd.DataFrame(columns=cols,data = data )
df

,startdate,enddate,value
0,2018-06-02 18:59:25 +0300,2018-06-02 19:06:52 +0300,86
1,2018-06-02 19:37:34 +0300,2018-06-02 19:44:40 +0300,66
2,2018-06-02 21:01:04 +0300,2018-06-02 21:07:09 +0300,8
3,2018-06-02 21:40:05 +0300,2018-06-02 21:47:47 +0300,9
4,2018-06-02 21:55:05 +0300,2018-06-02 22:05:00 +0300,194
...,...,...,...
31111,2021-09-19 17:24:13 +0300,2021-09-19 17:24:16 +0300,12
31112,2021-09-19 17:49:35 +0300,2021-09-19 17:50:39 +0300,24
31113,2021-09-19 18:19:58 +0300,2021-09-19 18:21:28 +0300,91
31114,2021-09-19 18:45:40 +0300,2021-09-19 18:48:14 +0300,121


In [25]:
#parse start time and end time
df['startdate'] = pd.to_datetime(df['startdate'])
df['enddate'] = pd.to_datetime(df['enddate'])
df['value'] =pd.to_numeric(df['value'])
#calculate the interval of each record
df['duration'] = (df['enddate']-df['startdate']).astype('timedelta64[s]')

#extract date only for grouping, start hour and end hour for interpolation 
df['date'] = df['startdate'].dt.date
df['start_hour'] = df['startdate'].dt.hour
df['end_hour'] = df['enddate'].dt.hour

#calculate days between records to check for gaps
df.sort_values('startdate',inplace=True)
df['days_gap'] = (df['startdate'].diff()).astype('timedelta64[D]')
df['days_gap'] = df['days_gap'].fillna(0)





#check need for interpolation
print("There are ",df[df['end_hour']!=df['start_hour']].shape[0], 'records that need interpolation')
print("There are ",df[(df['end_hour']==0) & (df['start_hour']==23)].shape[0],' records that ended on next day ')
print(max(df['days_gap'])," maximum days gap in data") 



There are  3633 records that need interpolation
There are  190  records that ended on next day 
0.0  maximum days gap in data


In [26]:
df.groupby('date').count()

,startdate,enddate,value,duration,start_hour,end_hour,days_gap
date,,,,,,,
2018-06-02,13,13,13,13,13,13,13
2018-06-03,32,32,32,32,32,32,32
2018-06-04,38,38,38,38,38,38,38
2018-06-05,32,32,32,32,32,32,32
2018-06-06,36,36,36,36,36,36,36
...,...,...,...,...,...,...,...
2021-09-16,24,24,24,24,24,24,24
2021-09-17,23,23,23,23,23,23,23
2021-09-18,30,30,30,30,30,30,30


In [27]:
#min_date = min(df['date']).strftime('%Y-%m-%d')
#max_date = max(df['date']).strftime('%Y-%m-%d')
#print(min_date,max_date)
#max_date
#date_range = pd.date_range(start=min_date,end=max_date)


In [28]:
df.head()

,startdate,enddate,value,duration,date,start_hour,end_hour,days_gap
0,2018-06-02 18:59:25+03:00,2018-06-02 19:06:52+03:00,86,447.0,2018-06-02,18,19,0.0
1,2018-06-02 19:37:34+03:00,2018-06-02 19:44:40+03:00,66,426.0,2018-06-02,19,19,0.0
2,2018-06-02 21:01:04+03:00,2018-06-02 21:07:09+03:00,8,365.0,2018-06-02,21,21,0.0
3,2018-06-02 21:40:05+03:00,2018-06-02 21:47:47+03:00,9,462.0,2018-06-02,21,21,0.0
4,2018-06-02 21:55:05+03:00,2018-06-02 22:05:00+03:00,194,595.0,2018-06-02,21,22,0.0


In [30]:
df_daily = df.groupby("date")["value"].sum().reset_index()
df_daily

,date,value
0,2018-06-02,3140
1,2018-06-03,9810
2,2018-06-04,6551
3,2018-06-05,6107
4,2018-06-06,2569
...,...,...
1202,2021-09-16,4454
1203,2021-09-17,3702
1204,2021-09-18,3939
1205,2021-09-19,2847


In [20]:
df_daily = df_daily[['date','value']]
df_daily.columns= ['Date','Steps']
df_daily.to_csv('steps.csv')